# Assignment: MTA Arrival Clock

In the "MTA Arrival Clock" case study, we have created an alternative design to visualize the arrival time information that looks at the following:

<div><img src="https://user-images.githubusercontent.com/3606672/246465520-5b7cad68-4722-4192-ad69-d0916c8f9cea.svg" width=400/></div>

As noted in the case study, it has the following drawbacks:

* Only trains arriving within the next 10 min are displayed.
* When two or more trains have the same arrival time, the subway line symbols and corresponding annotations will overlap.

## Task:

In this assignment, your task is to come up with a new design that addresses at least one of the two aforementioned drawbacks. You can either use the existing dot plot design or come up with a complete new design. The requirements of the design is the following:

* The dimension of the visualization is fixed (833x200).
* The design should make it easy to extract the arrival time information for each of the upcoming trains.
* The arrival time information should be legible from afar.
* Be mindful about utilizing the available spaces.

Please use this notebook for this assignment.

In [178]:
import altair as alt
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "line": ["F", "F", "E", "M"],
    "arrival_time": [3, 3, 6, 0], 
    "destination": ["Jamaica-179 St", "Kings Hwy", "Jamaica-179 St", "Forest Hills-71 Av"]})
line_color = pd.DataFrame({
    "line": ["E", "F", "M"],
    "color": ["#0039a6", "#ff6319", "#ff6319"]
})

In [179]:
df = df.sort_values(by='arrival_time', ascending=True)

In [180]:
# Create the dataframe of the background dots representing the time
space_factor = min(1 + len(df) / 16, 2)
df['x'] = np.arange(len(df))*space_factor
# Restrict the scale of the graph to the domain of the dots
x_scale = alt.Scale(domain=[-1, df['x'].max() + 1])
# Create background chart
background = alt.Chart(df).mark_circle(size=100, color='gray').encode(
    x=alt.X('x:Q', axis=None, scale=x_scale)
    ).properties(width=833, height=200)
background

alt.Chart(...)

In [181]:
arrival_time = alt.Chart(df).mark_circle(size=2500, opacity=1).encode(
    x=alt.X('x:Q', axis=None, scale=x_scale), 
    color=alt.Color('color:N', scale=None)
).transform_lookup(
    lookup='line', 
    from_=alt.LookupData(line_color, 'line', ['color'])
)
background + arrival_time

alt.LayerChart(...)

In [182]:
line_label = alt.Chart(df).mark_text(color='white', fontSize=30, font='Helvetica', fontStyle='bold').encode(
    x=alt.X('x:Q', axis=None, scale=x_scale), text='line:N'
)
background + arrival_time + line_label

alt.LayerChart(...)

In [183]:
dir_label = alt.Chart().mark_text(color='gray', fontSize=50, font='Helvetica', fontStyle='bold').encode(
    x=alt.value(20), y=alt.value(102), text=alt.value('❮')
)
background + dir_label + arrival_time + line_label

alt.LayerChart(...)

In [184]:
eta_text = alt.Chart(df).mark_text(color='black', fontSize=20, font='Helvetica').encode(
    x=alt.X('x:Q', axis=None, scale=x_scale), 
    y=alt.value(140), 
    text=alt.condition(alt.datum.arrival_time == 0, alt.value('arriving'), 'time:N')
).transform_calculate(time="datum.arrival_time + ' min'")
background + dir_label + eta_text + arrival_time + line_label

alt.LayerChart(...)

In [185]:
destination_label = alt.Chart(df).mark_text(color='black', fontSize=16, font='Helvetica').encode(
    x=alt.X('x:Q', axis=None, scale=x_scale),
    y=alt.value(60),
    text='destination:N'
)
background + dir_label + eta_text + destination_label + arrival_time + line_label

alt.LayerChart(...)

In [186]:
(background + dir_label + eta_text + destination_label + arrival_time + line_label).save('chart.png')